# Species distribution modeling with ensemble model

## 1. Setup

### 1.1 Install dependencies

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score, label_ranking_loss, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

## 2. Data loading

### 2.1 Downloading data

In [2]:
os.getcwd()
# If not set to Project, change current working directory to Project

os.chdir("../")  # move one level up
print(os.getcwd())

/Users/fionajetzer/Desktop/IPEO/Project/Project


In [3]:
import sys
sys.path.append("Test")  # the folder containing geoplant_dataset.py

In [4]:
from geoplant_dataset import GeoPlantDataset
from multiprocessing import Pool

In [5]:
env_train = pd.read_csv("data/env_variables_training.csv")
env_test  = pd.read_csv("data/env_variables_test.csv")

ts_train  = pd.read_csv("data/landsat_timeseries_training.csv")
ts_test   = pd.read_csv("data/landsat_timeseries_test.csv")

# ---- Numpy files ----
img_train = np.load("data/satellite_patches_training.npy")
img_test  = np.load("data/satellite_patches_test.npy")

species_train = np.load("data/species_data_training.npy")
species_test  = np.load("data/species_data_test.npy")

print("Training shapes:")
print("Env:", env_train.shape)
print("Time-series:", ts_train.shape)
print("Images:", img_train.shape)
print("Labels:", species_train.shape)

Training shapes:
Env: (5000, 22)
Time-series: (5000, 161)
Images: (5000, 3, 128, 128)
Labels: (5000, 342)


### 2.2 Reshaping and removing columns

In [6]:
# Extract coordinates
train_lons = env_train.iloc[:, 1]
train_lats = env_train.iloc[:, 2]
test_lons = env_test.iloc[:, 1]
test_lats = env_test.iloc[:, 2]

env_train = env_train.values.astype(np.float32)
env_test  = env_test.values.astype(np.float32)

env_train = env_train[:, 3:]
env_test  = env_test[:, 3:]

mean_env = env_train.mean(axis=0)
std_env = env_train.std(axis=0)
env_train = (env_train - mean_env) / std_env
env_test = (env_test - mean_env) / std_env


ts_train = ts_train.values.astype(np.float32)
ts_test  = ts_test.values.astype(np.float32)

ts_train = ts_train[:, 1:161].reshape(-1, 40, 4)
ts_test  = ts_test[:, 1:161].reshape(-1, 40, 4)

mean_ts = ts_train.mean(axis=(0, 1))
std_ts = ts_train.std(axis=(0, 1))
ts_train = (ts_train - mean_ts) / std_ts
ts_test = (ts_test - mean_ts) / std_ts


img_train = img_train.astype(np.float32)/255.0
img_test  = img_test.astype(np.float32)/255.0

species_train = species_train.astype(np.float32)
species_test  = species_test.astype(np.float32)

### 2.3 Defining dataset class

### 2.4 Creating dataset

In [31]:
train_dataset = GeoPlantDataset(
    images=img_train,
    timeseries=ts_train,
    tabular=env_train,
    labels=species_train,
    split='train',
    val_split=0.2
)

val_dataset = GeoPlantDataset(
    images=img_train,
    timeseries=ts_train,
    tabular=env_train,
    labels=species_train,
    split='val',
    val_split=0.2
)

test_dataset = GeoPlantDataset(
    images=np.expand_dims(img_test[100], axis=0),      # shape (1, C, H, W)
    timeseries=np.expand_dims(ts_test[100], axis=0),   # shape (1, T)
    tabular=np.expand_dims(env_test[100], axis=0),     # shape (1, F)
    labels=np.expand_dims(species_test[100], axis=0),  # shape (1, num_species)
    split='test'
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)}")
print(f"Val samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Train samples: 4000
Val samples: 1000
Test samples: 1


## 3. Multimodal species distribution model

In [19]:
# ------------------------------------------------------------
# 3.1 TABULAR ENCODER (climate variables)
# ------------------------------------------------------------
class TabularEncoder(nn.Module):
    def __init__(self, in_features=19, out_features=342):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(in_features, 64),
          nn.BatchNorm1d(64),
          nn.ReLU(),
          nn.Dropout(0.1),

          nn.Linear(64, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
          nn.Dropout(0.2),

          nn.Linear(128, 256),
          nn.BatchNorm1d(256),
          nn.ReLU(),
          nn.Dropout(0.3),

          nn.Linear(256, 512),
          nn.BatchNorm1d(512),
          nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


# ------------------------------------------------------------
# 3.2 TEMPORAL ENCODER (Landsat quarterly 10-year series)
#      Input shape: (batch, T=40, C=4)
# ------------------------------------------------------------
class TimeseriesEncoder(nn.Module):
    def __init__(self, in_channels=4, out_features=342):
        super().__init__()

        # Convolutional Backbone
        self.features = nn.Sequential(
            self._make_layer(in_channels, 32),
            self._make_layer(32, 64),
            self._make_layer(64, 128),
            self._make_layer(128, 256),
            self._make_layer(256, 512),
        )

        # Pooling
        self.pool = nn.AdaptiveAvgPool1d(1)

        # Dropout
        self.dropout = nn.Dropout(0.2)


    def _make_layer(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv1d(in_c, out_c, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_c), # Added for stability
            nn.ReLU()
        )


    def forward(self, ts):
        # ts shape: (B, T, C) -> (B, C, T)
        ts = ts.permute(0, 2, 1)

        x = self.features(ts)

        # Global Average Pool: (B, 512, T) -> (B, 512)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return x


# ------------------------------------------------------------
# 3.3 IMAGE ENCODER (Sentinel-2 RGB patches)
#      Uses pretrained ResNet34 but outputs a 256-dim feature
# ------------------------------------------------------------
from torchvision.models import resnet34

class ImageEncoder(nn.Module):
    def __init__(self, num_species=342):
        super().__init__()
        # Load weights suited for natural images
        base = resnet34(weights="DEFAULT")

        # Keep everything except final FC layer
        self.backbone = nn.Sequential(*list(base.children())[:-1])

        # Add FC
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

    def forward(self, img):
        features = self.backbone(img)
        x = self.head(features)
        return x

# ------------------------------------------------------------
# 3.4 FUSION MODEL
# ------------------------------------------------------------
class MultimodalSpeciesModel(nn.Module):
    def __init__(self, num_species=342):
        super().__init__()
        self.tabular = TabularEncoder()
        self.temporal = TimeseriesEncoder()
        self.image   = ImageEncoder()

        fusion_dim = 3*512
        self.fusion = nn.Sequential(
            nn.Linear(fusion_dim,1536),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1536,1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_species)
        )

    def forward(self, img, ts, tab):
        f_img = self.image(img)
        f_ts  = self.temporal(ts)
        f_tab = self.tabular(tab)

        fused = torch.cat([f_img, f_ts, f_tab], dim=1)
        logits = self.fusion(fused)
        return logits


# instantiate the model
model = MultimodalSpeciesModel()
print("Multimodal model created with parameters:", sum(p.numel() for p in model.parameters()))


Multimodal model created with parameters: 26884022


In [20]:
model_weights = "Models/Multimodal/Multimodal_model.pth"

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate encoders
model = MultimodalSpeciesModel()

# Load weights
model.load_state_dict(torch.load(model_weights, map_location=device, weights_only=True))

model = model.to(device)

In [21]:
def predict(loader, model, device):
    model.eval()
    
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for img, ts, env, y in loader:
            img = img.to(device).float()
            ts  = ts.to(device).float()
            env = env.to(device).float()
            y = y.to(device).float()

            # Model outputs (probabilities)
            batch_probs = torch.sigmoid(model(img, ts, env))
            
            # Store batch results
            all_probs.append(batch_probs.cpu())   # move to CPU
            all_labels.append(y.cpu())            # also store labels

    # Concatenate all batches
    all_probs = torch.cat(all_probs, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    print("Probs shape:", all_probs.shape)
    print("Labels shape:", all_labels.shape)

    return all_probs, all_labels

In [32]:
test_probs, test_labels = predict(model=model, loader=test_loader, device=device)

Probs shape: torch.Size([1, 342])
Labels shape: torch.Size([1, 342])


In [23]:
full_train_data = GeoPlantDataset(tabular=env_train,images=img_train, timeseries=ts_train, labels=species_train, split=None)
train_loader = DataLoader(full_train_data, batch_size=32, shuffle=True, num_workers=2)

train_pred, train_label = predict(train_loader,
                     model,
                     device=device)

Probs shape: torch.Size([5000, 342])
Labels shape: torch.Size([5000, 342])


In [24]:
def find_optimal_thresholds(y_true, probs):
    """
    Finds the best threshold for each of the 342 species.
    
    Args:
        y_true: Ground truth labels (N, 342) -> torch.Tensor or np.ndarray
        probs: Training predictions (N, 342) -> torch.Tensor or np.ndarray

    Returns:
        best_thresholds: np.ndarray of shape (342,) with optimized thresholds
    """

    # Convert tensors to NumPy if needed
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.cpu().numpy()
    if isinstance(probs, torch.Tensor):
        probs = probs.cpu().numpy()

    num_species = y_true.shape[1]
    best_thresholds = np.zeros(num_species)
    thresholds = np.linspace(0, 1, 100)

    print("Optimizing thresholds for 342 species...")

    for i in range(num_species):
        species_labels = y_true[:, i]
        species_probs = probs[:, i]

        # Default threshold if no positive samples
        if species_labels.sum() == 0:
            best_thresholds[i] = 0.5
            continue

        best_fbeta = -1
        best_thresh = 0.5

        for t in thresholds:
            preds = (species_probs >= t).astype(int)
            score = fbeta_score(species_labels, preds, beta=0.5, zero_division=0)

            if score > best_fbeta:
                best_fbeta = score
                best_thresh = t

        best_thresholds[i] = best_thresh

    print("Optimization complete.")
    return best_thresholds

In [25]:
opt_thresholds = find_optimal_thresholds(train_label, train_pred)

Optimizing thresholds for 342 species...
Optimization complete.


In [33]:
binary_preds = (test_probs.cpu().numpy() >= opt_thresholds).astype(int)

In [36]:
idx_pred = np.nonzero(binary_preds)[1]
idx_label = np.nonzero(test_labels)[1]
print("Species index predicted present:", idx_pred)
print("Species index labeled present:", idx_label.numpy())

Species index predicted present: [ 15  30  56  81  94  96 113 128 135 143 187 212 252 263 268 272 282 304
 315 338]
Species index labeled present: [ 0 30]
